In [186]:
from IPython.core.display import display, HTML

import plotly.graph_objects as go
from plotly.offline import plot

import pandas as pd
from pandas_datareader import data as web 


from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

from matplotlib import dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mpl_dates

import mplfinance as fplt
from mpl_finance import candlestick_ohlc

fig
df

def getEndDate(tipo, quantidade):
    
    if(tipo.lower()=="m"):
        data_fim = datetime.today() - relativedelta(months=int(quantidade))      
    

    return data_fim.strftime('%m-%d-%Y')

def getData(acao,start, end):
    df = web.DataReader(f'{acao}', data_source='yahoo', start=start, end=end)

    df.reset_index(inplace=True,drop=False)
    
    df['MA9'] = df.Close.rolling(9).mean()
    df['MA21'] = df.Close.rolling(21).mean()
    df['MA200'] = df.Close.rolling(200).mean()
    
    df["MME9"] = df["Adj Close"].ewm(span=9, min_periods=9).mean()
    df["MME21"] = df["Adj Close"].ewm(span=21, min_periods=21).mean()

#    display(df)
    
    return df
    

def plotaCandleStickPlotLy(acao, start, end):
        
        df = getData(acao, start, end)
        
    
        fig = go.FigureWidget(data=[go.Candlestick(x=df['Date'],
                        open=df['Open'],
                        high=df['High'],
                        low=df['Low'],
                        close=df['Close']),
                        go.Scatter(name="MME9", x=df.Date, y=df.MA9, line=dict(color='white', width=1 )),
                        go.Scatter(name="MME21",x=df.Date, y=df.MA21, line=dict(color='red', width=1, dash="dash")),
                        go.Scatter(name="SMA200",x=df.Date, y=df.MA200, line=dict(color='purple', width=1, dash="dash"))
                             ]                       
                       )
        
        fig.update_xaxes(
            rangebreaks=[
                dict(bounds=["sat", "mon"]), #hide weekends                
            ]
        )
        
        fig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                         dict(count=7,
                             label="1w",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(count=1,
                             label="YTD",
                             step="year",
                             stepmode="todate"),
                        dict(count=1,
                             label="1y",
                             step="year",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible= True 
                ),
                type="date"
            ),
#             paper_bgcolor="LightSteelBlue",
            height=700,
#             font_color="navy",  
            template="plotly_dark",
            title=f'{acao} {start} {end}',
            font_family="Arial Black",                      
            legend_title_font_color="green",
            yaxis = dict(fixedrange = False,autorange= True, side="right", tickformat = ".2f"),         
        )
     
        fig.update_layout(hoverdistance=10)
        fig.update_layout(hovermode="x")
        
        
        fig.show()
        
def plotaCandleStickPlotLyOffline(acao, start, end):
    
        df = getData(acao, start, end)
        
        o = df['Open'].astype(float)
        h = df['High'].astype(float)
        l = df['Low'].astype(float)
        c = df['Close'].astype(float)
        
        trace = go.Candlestick(
                    x=df['Date'],
                    open=o,
                    high=h,
                    low=l,
                    close=c
                    )
        data = [trace]

        plot(data, filename=f'{acao}_{start}_{end}.html')


        
def zoom(layout, xrange):
    print("zooming")
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
    fig.update_layout(title = "olá mundo")
        


display(HTML("<style>.container { width:100% !important; }</style>"))

end = date.today().strftime('%m-%d-%Y')

start = getEndDate("m", 36)

stocks = ["AAPL"]

for stock in stocks:
    plotaCandleStickPlotLy(stock, start, end)




